In [1]:

# Now, we will focus on feature extraction and engineering in order to ready the
# data for the clustering algorithm run. We instantiate the Spark Context and read
# the Twitter dataset into a Spark dataframe. We will then successively tokenize the tweet text data, 
# apply a hashing Term frequency algorithm to the tokens, 
# and  nally apply the Inverse Document Frequency algorithm and rescale the data. 
# The code is as follows:


import pandas as pd
from pyspark import SQLContext

# CSV to Pandas DF
csv_in = "/home/carl/spark/examples/carl_Spark/data/mllib/twtr15053001-5.csv"
pddf_in = pd.read_csv(csv_in, sep=',', encoding='utf-8')
sqlContext = SQLContext(sc)

# Pandas DF to Spark DF
spdf_02 = sqlContext.createDataFrame(pddf_in[['id', 'user_id', 'user_name']])

spdf_02.show()

+----------+----------------+--------------+
|        id|         user_id|     user_name|
+----------+----------------+--------------+
|3168398228|Who What Wear AU|Kendall Jenner|
+----------+----------------+--------------+



In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="user_name", outputCol="tokens")
tokensData = tokenizer.transform(spdf_02)


In [5]:
# Applying Hashing TF to the tokens
hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=2000)
featuresData = hashingTF.transform(tokensData)

featuresData.take(1)

[Row(id=3168398228, user_id=u'Who What Wear AU', user_name=u'Kendall Jenner', tokens=[u'kendall', u'jenner'], rawFeatures=SparseVector(2000, {203: 1.0, 1234: 1.0}))]

In [6]:
# Applying IDF(Inverse Document Frequency) to the raw features and rescale the data
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featuresData)
rescaledData = idfModel.transform(featuresData)

for features in rescaledData.select('features').take(3):
    print(features)

rescaledData.take(2)

Row(features=SparseVector(2000, {203: 0.0, 1234: 0.0}))


[Row(id=3168398228, user_id=u'Who What Wear AU', user_name=u'Kendall Jenner', tokens=[u'kendall', u'jenner'], rawFeatures=SparseVector(2000, {203: 1.0, 1234: 1.0}), features=SparseVector(2000, {203: 0.0, 1234: 0.0}))]

In [7]:
rs_pddf = rescaledData.toPandas()
rs_pddf.count()

id             1
user_id        1
user_name      1
tokens         1
rawFeatures    1
features       1
dtype: int64

In [8]:
feat_lst = rs_pddf.features.tolist()
feat_lst

[SparseVector(2000, {203: 0.0, 1234: 0.0})]

In [9]:
# We will use the K-Means algorithm against the Twitter dataset. 
# As an unlabeled and shuf ed bag of tweets, we want to see if the Apache Spark tweets are grouped in a single cluster. 
# From the previous steps, 
# the TF-IDF sparse vector of features is converted into an RDD that will be the input to the Spark MLlib program.
# We initialize the K-Means model with 5 clusters, 10 iterations of 10 runs:

from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

in_Data = sc.parallelize(feat_lst)

in_Data.take(3)




[SparseVector(2000, {203: 0.0, 1234: 0.0})]

In [10]:
in_Data.count()
# parsedData = in_Data.map(lambda line: array([Vendor(line)]))

1

In [11]:
# Build the model
clusters = KMeans.train(in_Data, 2, maxIterations=10, runs=10, initializationMode="random")

# Evaluate clustering by computing within Set Sum of Squared Errors

/home/carl/spark/python/pyspark/mllib/clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.
  warnings.warn("The param `runs` has no effect since Spark 2.0.0.")


Py4JJavaError: An error occurred while calling o131.trainKMeansModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/carl/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/home/carl/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/carl/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/carl/spark/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 80, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(l))
TypeError: Cannot convert type <class 'pyspark.ml.linalg.SparseVector'> into Vector

	at org.a

In [ ]:
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([xx*2 for x in (point - center)]))

WSSSE = in_Data.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

In [10]:
print(sc)